In [1]:
import pandas as pd
f = open('核心期刊数据.csv')
df = pd.read_csv(f)
df = df.dropna().reset_index(drop=True)
#如果编辑解释器不是jupyter-notebook或命令行，用print函数包裹下面的代码
df.head()

,期刊名,出版文献量,综合影响因子,复合影响因子,基金论文比,篇均他引,篇均被引,h指数,引用刊数,总被引频次,篇均引文数
0,中国图书馆学报,113,5.189,6.358,0.5965,10.50,11.34,20,224,4109,38.47
1,大学图书馆学报,197,2.000,2.205,0.3496,5.95,6.32,17,256,3027,20.44
2,图书情报工作,860,2.283,2.769,0.4865,4.84,5.30,23,563,11818,18.30
3,情报学报,223,0.958,1.150,0.7463,2.59,3.14,8,139,2215,30.73
4,图书情报知识,180,2.646,3.263,0.6512,6.29,6.76,16,294,2373,29.13


In [2]:
class EmtropyMethod:
    def __init__(self, index, positive, negative, row_name):
        if len(index) != len(row_name):
            raise Exception('数据指标行数与行名称数不符')
        if sorted(index.columns) != sorted(positive+negative):
            raise Exception('正项指标加负向指标不等于数据指标的条目数')

        self.index = index.copy().astype('float64')
        self.positive = positive
        self.negative = negative
        self.row_name = row_name.copy()
    
    def uniform(self):
        uniform_mat = self.index.copy()
        min_index = {column:min(uniform_mat[column]) for column in uniform_mat.columns}
        max_index = {column:max(uniform_mat[column]) for column in uniform_mat.columns}
        for i in range(len(uniform_mat)):
            for column in uniform_mat.columns:
                if column in self.negative:
                    uniform_mat[column][i] = (uniform_mat[column][i] - min_index[column]) / (max_index[column] - min_index[column])
                else:
                    uniform_mat[column][i] = (max_index[column] - uniform_mat[column][i]) / (max_index[column] - min_index[column])

        self.uniform_mat = uniform_mat
        return self.uniform_mat
    def cal_finalscore()
    def calc_probability(self):
        try:
            p_mat = self.uniform_mat.copy()
        except AttributeError:
            raise Exception('你还没进行归一化处理，请先调用uniform方法')
        for column in p_mat.columns:
            sigma_x_1_n_j = sum(p_mat[column])
            # 为了取对数计算时不出现无穷,将比重为0的值修改为1e-6
            p_mat[column] = p_mat[column].apply(lambda x_i_j: x_i_j / sigma_x_1_n_j if x_i_j / sigma_x_1_n_j != 0 else 1e-6)

        self.p_mat = p_mat
        return p_mat
    def calc_emtropy(self):
        try:
            self.p_mat.head(0)
        except AttributeError:
            raise Exception('你还没计算比重，请先调用calc_probability方法')

        import numpy as np
        e_j  = -(1 / np.log(len(self.p_mat)+1)) * np.array([sum([pij*np.log(pij) for pij in self.p_mat[column]]) for column in self.p_mat.columns])
        ejs = pd.Series(e_j, index=self.p_mat.columns, name='指标的熵值')

        self.emtropy_series = ejs
        return self.emtropy_series
    def calc_emtropy_redundancy(self):
        try:
            self.d_series = 1 - self.emtropy_series
            self.d_series.name = '信息熵冗余度'
        except AttributeError:
            raise Exception('你还没计算信息熵，请先调用calc_emtropy方法')

        return self.d_series
    def calc_Weight(self):
        self.uniform()
        self.calc_probability()
        self.calc_emtropy()
        self.calc_emtropy_redundancy()
        self.Weight = self.d_series / sum(self.d_series)
        self.Weight.name = '权值'
        return self.Weight

In [3]:
# 归一化矩阵
indexs = ['出版文献量', '综合影响因子', '复合影响因子', '基金论文比', '篇均他引', '篇均被引', 'h指数', '引用刊数', '总被引频次', '篇均引文数']
positive = indexs
negative = []
journal_name = df['期刊名']
index = df[indexs]
em = EmtropyMethod(index, negative, positive, journal_name)
em.uniform()

,出版文献量,综合影响因子,复合影响因子,基金论文比,篇均他引,篇均被引,h指数,引用刊数,总被引频次,篇均引文数
0,0.000000,1.000000,1.000000,0.527226,1.000000,1.000000,0.800000,0.194399,0.256103,1.000000
1,0.112450,0.256643,0.202573,0.000000,0.424779,0.387805,0.600000,0.247117,0.151694,0.352836
2,1.000000,0.322611,0.310868,0.292334,0.284450,0.263415,1.000000,0.752883,1.000000,0.276023
3,0.147256,0.013753,0.000000,0.847107,0.000000,0.000000,0.000000,0.054366,0.073338,0.722182
4,0.089692,0.407226,0.405722,0.644032,0.467762,0.441463,0.533333,0.309720,0.088584,0.664752
5,0.738956,0.146154,0.191052,1.000000,0.246523,0.231707,0.733333,0.733114,0.455274,0.367911
6,0.066934,0.203730,0.217742,0.342088,0.237674,0.202439,0.266667,0.154860,0.000000,0.254846
7,0.917001,0.125175,0.177227,0.910741,0.207332,0.197561,0.666667,0.901153,0.743221,0.426777
8,0.155288,0.304429,0.283986,0.486227,0.439949,0.397561,0.600000,0.314662,0.163080,0.304379
9,0.870147,0.080886,0.132104,0.794790,0.299621,0.254878,0.733333,1.000000,0.536910,0.406676


In [4]:
# 计算每个值在指标所有值中的比重
em.calc_probability()

,出版文献量,综合影响因子,复合影响因子,基金论文比,篇均他引,篇均被引,h指数,引用刊数,总被引频次,篇均引文数
0,0.000001,0.279206,0.275381,0.058755,0.187708,0.209719,0.078947,0.026928,0.050991,0.147462
1,0.015490,0.071656,0.055785,0.000001,0.079734,0.081330,0.059211,0.034231,0.030203,0.052030
2,0.137747,0.090075,0.085607,0.032578,0.053393,0.055243,0.098684,0.104290,0.199105,0.040703
3,0.020284,0.003840,0.000001,0.094403,0.000001,0.000001,0.000001,0.007531,0.014602,0.106494
4,0.012355,0.113700,0.111728,0.071772,0.087803,0.092583,0.052632,0.042903,0.017638,0.098026
5,0.101789,0.040807,0.052612,0.111442,0.046274,0.048593,0.072368,0.101552,0.090647,0.054253
6,0.009220,0.056883,0.059962,0.038123,0.044613,0.042455,0.026316,0.021451,0.000001,0.037580
7,0.126314,0.034950,0.048805,0.101494,0.038918,0.041432,0.065789,0.124829,0.147979,0.062933
8,0.021390,0.084998,0.078204,0.054186,0.082582,0.083376,0.059211,0.043587,0.032470,0.044884
9,0.119860,0.022584,0.036379,0.088573,0.056241,0.053453,0.072368,0.138521,0.106901,0.059969


In [6]:
# 计算指标的熵
em.calc_emtropy()

出版文献量     0.860610
综合影响因子    0.809863
复合影响因子    0.826195
基金论文比     0.899167
篇均他引      0.906890
篇均被引      0.891418
h指数       0.937784
引用刊数      0.886373
总被引频次     0.853814
篇均引文数     0.918961
Name: 指标的熵值, dtype: float64

In [7]:
# 计算信息熵冗余度(1-熵值)
em.calc_emtropy_redundancy()

出版文献量     0.139390
综合影响因子    0.190137
复合影响因子    0.173805
基金论文比     0.100833
篇均他引      0.093110
篇均被引      0.108582
h指数       0.062216
引用刊数      0.113627
总被引频次     0.146186
篇均引文数     0.081039
Name: 信息熵冗余度, dtype: float64

In [8]:
# 计算权重(将信息熵冗余度归一化)
em.calc_Weight()

出版文献量     0.115301
综合影响因子    0.157278
复合影响因子    0.143768
基金论文比     0.083407
篇均他引      0.077019
篇均被引      0.089817
h指数       0.051464
引用刊数      0.093990
总被引频次     0.120923
篇均引文数     0.067034
Name: 权值, dtype: float64

In [15]:
em_weight = [0.115301, 0.157278, 0.143768, 0.083407, 0.077019, 0.089817, 0.051464, 0.093990, 0.120923, 0.067034]
F = [[-0.066, 0.280, -0.022],
[0.216, -0.012, -0.020],
[0.209, -0.003, 0.030],
[-0.068, 0.029, 0.646],
[0.219, 0.006, -0.088],
[0.218, 0.000, -0.050],
[0.115, 0.244, -0.221],
[-0.019, 0.276, -0.057],
[0.003, 0.269, 0.034],
[0.107, -0.084, 0.458]]
weight = [0.47123, 0.3137, 0.13607]

In [16]:
import numpy as np
weight = np.array(weight)
indicator_weight = []
for i in range(10):
    em_indicator = em_weight[i]
    Fi = np.array(F[i])
    weight_i = em_indicator * np.dot(Fi, weight)
    indicator_weight.append(weight_i)

In [18]:
# 计算指标的最终权重
ejs = pd.Series(indicator_weight, index=indexs, name='指标的熵值')
ejs

出版文献量     0.006196
综合影响因子    0.014989
复合影响因子    0.014611
基金论文比     0.005418
篇均他引      0.007171
篇均被引      0.008616
h指数       0.005181
引用刊数      0.006567
总被引频次     0.010935
篇均引文数     0.005791
Name: 指标的熵值, dtype: float64

In [20]:
writer = pd.ExcelWriter('normalized.xlsx')
em.uniform().to_excel(writer, 'Sheet1')
writer.save()